# Very basic `qsub`, `qstat`, `qdel` Python wrapper

## Basic code

In [33]:
# qsub_utils

import os
import subprocess
import platform

PBS_CONFIGURATION=""

def setup_configuration(nodes='', stdout='', stderr=''):
    """
    Method to setup PBS configuration
    :param nodes: specify nodes with list of features. `-l nodes={nodes}`
    :param stdout: specify stdout file. `-o {stdout}`
    :param stderr: specify stderr file. `-e {stderr}`
    """
    global PBS_CONFIGURATION
    conf = "#PBS"
    if len(nodes) > 0:
        conf += " -l nodes=%s" % nodes        
    if len(stdout) > 0:
        conf += " -o %s" % stdout
    if len(stderr) > 0:
        conf += " -e %s" % stderr
        
    PBS_CONFIGURATION=conf

    
def write_launch_file(cmd_str, name, cwd='', env=''):    
    """
    Method to write a PBS launch file for qsub
    
    :param cmd_str: list of 
    """
    assert len(cmd_str) > 0, "Job command can't be empty"
    assert len(name) > 0, "Job name can't be empty"
    filename = os.path.join(cwd, "job_%s.launch" % name)
    with open(filename, 'w') as w:
        conf = PBS_CONFIGURATION
        conf += " -N %s" % name
        if len(cwd) > 0:
            conf += " -d %s" % cwd        
        w.write(conf + '\n')        
        if len(env) > 0:
            w.write(env + '\n')
        w.write(cmd_str)    
        
    return filename
    
    
def submit_job(cmd, name, cwd='', env=''):
    """
    Method to submit a job writing a launch file and using qsub
    `qsub job_{name}.launch`
    
    :param cmd:
    """
                    
    filename = write_launch_file(' '.join(cmd), name, cwd, env)    
    
    program = ['qsub', filename]
    process = subprocess.Popen(program, 
                               stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT,
                               close_fds=False if platform.system() == 'Windows' else True)
    job_id = None
    while process.poll() is None:
        time.sleep(0.01)
    
    job_id = process.stdout
    assert job_id is not None and len(job_id) > 0, "Failed to fetch job id from qsub"
    return job_id, filename
    

## Example

In [34]:
setup_configuration(nodes='1:knl7210:ram96gb', stdout='/home/u2459', stderr='/home/u2459')

caffe_help_cmd = [
    "caffe",
    "--help", 
]
job_id, filename = submit_job(caffe_help_cmd, 'help', env='export PATH=$PATH:/Users/vfomin/Documents/ML/caffe/build/tools/')

OSError: [Errno 2] No such file or directory

In [35]:
!cat {filename}

cat: None: No such file or directory
